In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from matplotlib import colors
import networkx as nx
from pyvis.network import Network


In [9]:
PATH = "../serial-reproduction-with-selection/analysis/data/rivka-necklace-rep-data/psynet/data/"
nodes = pd.read_csv(PATH + "node.csv", low_memory=False)
networks = pd.read_csv(PATH + "network.csv", low_memory=False)

In [10]:
# filter networks
network_data = networks
network_data = network_data[network_data["role"] == "experiment"]
network_data = network_data[network_data["failed"] == 'f']
network_data = network_data[network_data["trial_maker_id"] == 'graph_experiment']

experiment_net_id = list(network_data['id'].to_numpy())

# filter nodes
node_data = nodes
node_data = node_data[nodes["type"] == "graph_chain_node"]
node_data = node_data[node_data["failed"] == "f"]
node_data = node_data[node_data["network_id"].isin(experiment_net_id)]
node_data = node_data[["id", "network_id", "degree", "definition", "seed", "vertex_id", "dependent_vertex_ids"]]
node_data = node_data.sort_values(["network_id", "degree"])

In [42]:
# slice_graphs = []
# min_deg = int(node_data["degree"].min())
# max_deg = int(node_data["degree"].max())

# for deg in range(min_deg, max_deg + 1):
#     print(deg)

# ---- TODO: Generalize to each degree ---
# add nodes to new Graph
deg = node_data[node_data["degree"] == 1.0]
G = nx.Graph()
for node_id in deg["id"].values.tolist():
    v_id = deg[deg["id"] == node_id]["vertex_id"].values[0]
    G.add_node(node_id, vertex_id=v_id)

edges = {}
# add edges to the Graph 
for i, ser in deg.iterrows():
    node_id = ser["id"]

    # get dependent nodes 
    dependent_vertices = ser["dependent_vertex_ids"].strip('][').split(',')
    dependent_nodes = [deg[deg["vertex_id"] == float(v)] for v in dependent_vertices]
    dependent_nodes = [n["id"].values[0] for n in dependent_nodes]

    # add as edges
    edge_list = [(int(dependent_node), int(node_id)) for dependent_node in dependent_nodes]
    G.add_edges_from(edge_list)

In [32]:
# read infos, filter like nodes
infos = pd.read_csv(PATH + "info.csv")
info_data = infos
info_data = info_data[infos["type"] == "graph_chain_trial"]
info_data = info_data[info_data["failed"] == "f"]
info_data = info_data[["id", "creation_time", "details", "origin_id", "network_id", "participant_id"]] # TODO: Probably want more columns here 

info_data = info_data.sort_values(["network_id", "origin_id"])

In [33]:
# Add metadata from infos to each node
for n in G.nodes.items():
    n_id = n[0]
    row = info_data[info_data["origin_id"] == n_id]
    G.nodes[n_id]['creation_time'] = row["creation_time"].values[0]
    # TODO: Add other infos here 

In [41]:
# convert to PyVis and show
pyvis_net = Network('500px', '500px',notebook=True)
pyvis_net.from_nx(G)

# add labels
for n in pyvis_net.nodes:
   n['label'] = str(int(n['vertex_id'])) # gets changed to a float if you try to convert it in networkx for some rea

pyvis_net.show('slice.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
